# Why DNNClassifier can't find an apparent correlation?   

Why Tensorflow Estimator can't find the apparent coorelation when feature column width is simply enlarged and what is the correct usge?


In [ ]:
# Delete the existing network (Microsoft Windows)
!rd /q /s model\DNN

In [ ]:
import tensorflow as tf
import numpy as np

#### prepare feature and label

In [ ]:
feature_width = 20
np.random.seed(0)
x = np.random.rand(300,feature_width) # 
# it's good when feature_width is ~20 
# but when it comes simply to 2000 then I don't know how to handle it

In [ ]:
#  label is simply from a feature column
y = []
for i in range(len(x)):
    if x[i][0] >= 0.5:
        y.append(1)
    else:
        y.append(0)

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
xwidth = x.shape[-1]

In [ ]:
from sklearn import cross_validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size = 0.2)

#### prepare classifier

In [ ]:
my_feature_columns = [tf.feature_column.numeric_column("x", shape=[xwidth,])]  

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10,10],
    n_classes=2,
    dropout=0.2,
    model_dir="model/DNN"
    )

#### run training

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=None,
    batch_size=32,
    shuffle=True
    )

In [ ]:
classifier.train(input_fn=train_input_fn, steps=4000);

#### evaluate 

In [ ]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False
    )

In [ ]:
# Evaluate accuracy
evaluate = classifier.evaluate(input_fn=test_input_fn)
accuracy_score = evaluate["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

#### view the result 

When feature_width is ~20 it's good; when 2000 DNN crazily predicts all 1's or all 0's. How to handle that?

In [ ]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False, # so can be compared with label
    )

In [ ]:
preds = classifier.predict(pred_input_fn)

In [ ]:
predict = [i for i in preds];

In [ ]:
# np.argmax() wrapup the probabilities array
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html
predictions = [np.argmax(predict[i]['probabilities'],axis=-1) for i in range(len(x_test))]; 
print(predictions)
print(y_test)
print(sum(predictions))
print(sum(y_test))
print(np.array(predictions).shape)
print(np.array(y_test).shape)


In [ ]:
# compare predictions with labels
categories = [0,0,0,0]
for i in range(len(y_test)): 
    if int(predictions[i])==int(y_test[i]) and int(y_test[i])==0: categories[0]+=1; continue
    if int(predictions[i])!=int(y_test[i]) and int(y_test[i])==1: categories[1]+=1; continue
    if int(predictions[i])!=int(y_test[i]) and int(y_test[i])==0: categories[2]+=1; continue
    if int(predictions[i])==int(y_test[i]) and int(y_test[i])==1: categories[3]+=1; continue
print (categories) #  [ spectator, lost, missed, hit ]

In [ ]:
# check loss curve
!start tensorboard --logdir="model/DNN"